# Zero Shot

In [ ]:
import langchain 
from langchain import PromptTemplate, LLMChain
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=1, model_name="llama3-70b-8192",groq_api_key="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")

In [ ]:
msg = llm.invoke("Given the data in Data.csv, Which station recorded the highest PM2.5 level in March 2023 in India?")
print(msg.content)

# Zero Shot with csv file loading. 

In [14]:
import pandas as pd
df = pd.read_csv('Data.csv')
# Choose 1000 rows such that none of them contain a nan value
df = df.dropna().sample(1000)
df.to_csv('Time.csv')
df = pd.read_csv('Time.csv')
len(df)

1000

In [15]:
df.columns

Index(['Unnamed: 0', 'Timestamp', 'station', 'PM2.5', 'PM10', 'address',
       'city', 'latitude', 'longitude', 'state'],
      dtype='object')

In [47]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='Time.csv') #,metadata_columns=['Timestamp','station','address','city','latitude','longitude','state']
data = loader.load()

In [49]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=10,chunk_overlap = 0)
chunked_data = text_splitter.split_documents(data)

In [50]:
# chunked_data[0]

In [51]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [52]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_data,HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [91]:
db.search("Chattisgarh",search_type='mmr',k = 10)

[Document(page_content=': 699131\nTimestamp: 2020-07-25\nstation: H.B. Colony, Bhiwani - HSPCB\nPM2.5: 21.90238095238096\nPM10: 50.45813953488371\naddress: Mini Secretariat Office, 3rd Floor, H.B. Colony, Near Zoo, District. Bhiwani.\ncity: Bhiwani\nlatitude: 28.806223\nlongitude: 76.141105\nstate: Haryana', metadata={'source': 'Time.csv', 'row': 401}),
 Document(page_content=': 1233252\nTimestamp: 2023-04-16\nstation: Motilal Nehru NIT, Prayagraj - UPPCB\nPM2.5: 16.1696875\nPM10: 59.0092708333333\naddress: Barrister Mullah Colony, MNNIT Allahabad Campus, Teliarganj, Prayagraj, Uttar Pradesh\ncity: Prayagraj\nlatitude: 25.494\nlongitude: 81.863\nstate: Uttar Pradesh', metadata={'source': 'Time.csv', 'row': 505}),
 Document(page_content=': 1299006\nTimestamp: 2023-08-17\nstation: Civic Center, Bhilai - Bhilai Steel Plant\nPM2.5: 5.151847826086955\nPM10: 23.78804347826087\naddress: Back Side of Nehru Art Gallery, Civic center, Bhilai, Chattisgarh\ncity: Bhilai\nlatitude: 21.185571\nlongi

In [73]:
# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 10}
)

In [53]:
import langchain 
from langchain import PromptTemplate, LLMChain
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=1, model_name="llama3-70b-8192",groq_api_key="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")

In [74]:
from langchain.chains import ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever,return_source_documents=True)

In [75]:
import sys
chat_history = []
query = input()
# Ask question related CSV file like Which country has highest green gas?
result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Answer: I don't know. There is no data for Rajasthan in the provided context.

